### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# calculate totals
total_schools = len(school_data)
total_students = len(student_data)
total_budget = school_data["budget"].sum()

# calculate averages
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()

# add columns indicating whether the students passed or failed the reading and math tests
bins = [0,69.9,100]
bin_names = ['Fail', 'Pass']
student_data["read_PF"] = pd.cut(student_data["reading_score"], bins, labels=bin_names, include_lowest=True)
student_data["math_PF"] = pd.cut(student_data["math_score"], bins, labels=bin_names, include_lowest=True)

# calculate percentages
pass_read_pct = student_data["read_PF"].value_counts()["Pass"]/total_students*100
pass_math_pct = student_data["math_PF"].value_counts()["Pass"]/total_students*100
pass_read = student_data.loc[student_data["read_PF"] == 'Pass',:]
overall_pass_pct = pass_read["math_PF"].value_counts()["Pass"]/total_students*100

# create dataframe to neatly display calculations
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                                    "Total Students": [total_students],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math_score],
                                    "Average Reading Score": [avg_reading_score],
                                    "% Passing Math": [pass_math_pct],
                                    "% Passing Reading": [pass_read_pct],
                                    "% Overall Passing": [overall_pass_pct]
                                   })

# clean up formatting on output
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,.0f}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# extract the four columns from the school_data dataframe
school_summary_df = school_data[['school_name', 'type', 'size', 'budget']]
school_summary_df = school_summary_df.rename(columns={'school_name': 'School Name',
                                                     'type': 'School Type',
                                                     'size': 'Total Students',
                                                     'budget': 'Total School Budget',
                                                     })

# set the index to the school name & sort on that index
school_summary_df = school_summary_df.set_index('School Name')
school_summary_df = school_summary_df.sort_index()

# calculate the per student budget for each school
school_summary_df['Per Student Budget'] = school_summary_df['Total School Budget']/school_summary_df["Total Students"]

# add columns indicating whether the students passed or failed the reading and math tests
bins = [0,69.9,100]
bin_names = ['Fail', 'Pass']
school_data_complete["read_PF"] = pd.cut(school_data_complete["reading_score"], bins, labels=bin_names, include_lowest=True)
school_data_complete["math_PF"] = pd.cut(school_data_complete["math_score"], bins, labels=bin_names, include_lowest=True)

# calculate the average scores for all students at each school
comp_group_school = school_data_complete.groupby(['school_name'])
school_summary_df['Average Math Score'] = comp_group_school['math_score'].mean()
school_summary_df['Average Reading Score'] = comp_group_school['reading_score'].mean()

# calculate the percentage of students passing the math test at each school
math_series = comp_group_school['math_PF'].value_counts()
school_summary_df['% Passing Math'] = [math_series[x] for x in range(0, len(math_series),2)]/school_summary_df['Total Students']*100

# calculate the percentage of students passing the reading test at each school
read_series = comp_group_school['read_PF'].value_counts()
school_summary_df['% Passing Reading'] = [read_series[x] for x in range(0, len(read_series),2)]/school_summary_df['Total Students']*100
school_summary_df

# calculate the percentage of students passing the both the math and reading tests at each school
pass_read_comp = school_data_complete.loc[school_data_complete["read_PF"] == 'Pass',:]
overall_group_school = pass_read_comp.groupby(['school_name'])
overall_series = overall_group_school['math_PF'].value_counts()
school_summary_df['% Overall Passing'] = [overall_series[x] for x in range(0, len(overall_series),2)]/school_summary_df['Total Students']*100
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:.2f}%".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:.2f}%".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
top_perf_schools = school_summary_df.sort_values('% Overall Passing', ascending=False)
top_perf_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
bottom_perf_schools = school_summary_df.sort_values('% Overall Passing', ascending=True)
bottom_perf_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
ninth_df = school_data_complete.loc[school_data_complete['grade'] == '9th', :]
tenth_df = school_data_complete.loc[school_data_complete['grade'] == '10th', :]
eleventh_df = school_data_complete.loc[school_data_complete['grade'] == '11th', :]
twelfth_df = school_data_complete.loc[school_data_complete['grade'] == '12th', :]

math_scores_by_grade = pd.DataFrame(index=school_data['school_name'])
math_scores_by_grade = math_scores_by_grade.sort_index()
grouped_9th_math_df = ninth_df.groupby('school_name')
math_scores_by_grade['9th'] = grouped_9th_math_df['math_score'].mean()

grouped_10th_math_df = tenth_df.groupby('school_name')
math_scores_by_grade['10th'] = grouped_10th_math_df['math_score'].mean()

grouped_11th_math_df = eleventh_df.groupby('school_name')
math_scores_by_grade['11th'] = grouped_11th_math_df['math_score'].mean()

grouped_12th_math_df = twelfth_df.groupby('school_name')
math_scores_by_grade['12th'] = grouped_12th_math_df['math_score'].mean()

math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
read_scores_by_grade = pd.DataFrame(index=school_data['school_name'])
read_scores_by_grade = read_scores_by_grade.sort_index()
grouped_9th_read_df = ninth_df.groupby('school_name')
read_scores_by_grade['9th'] = grouped_9th_read_df['reading_score'].mean()

grouped_10th_read_df = tenth_df.groupby('school_name')
read_scores_by_grade['10th'] = grouped_10th_read_df['reading_score'].mean()

grouped_11th_read_df = eleventh_df.groupby('school_name')
read_scores_by_grade['11th'] = grouped_11th_read_df['reading_score'].mean()

grouped_12th_read_df = twelfth_df.groupby('school_name')
read_scores_by_grade['12th'] = grouped_12th_read_df['reading_score'].mean()

read_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [9]:
school_data['per_student_budget'] = school_data['budget']/school_data['size']

In [10]:
spend_bins = [0,584.999,629.999,644.999,675]
spend_bin_labels = ['<$584', '$585-629', '$630-644', '$645-675']
school_data["budget_groups"] = pd.cut(school_data["per_student_budget"], spend_bins, labels=spend_bin_labels, include_lowest=True)

# create budget group dataframes
budget_group1 = school_data.loc[school_data['budget_groups'] == '<$584',:]
budget_group2 = school_data.loc[school_data['budget_groups'] == '$585-629',:]
budget_group3 = school_data.loc[school_data['budget_groups'] == '$630-644',:]
budget_group4 = school_data.loc[school_data['budget_groups'] == '$645-675',:]

budget_group1 = budget_group1.reset_index(drop=True)
budget_group2 = budget_group2.reset_index(drop=True)
budget_group3 = budget_group3.reset_index(drop=True)
budget_group4 = budget_group4.reset_index(drop=True)

In [11]:
budget_group1_students = school_data_complete.loc[(school_data_complete['school_name'] == budget_group1['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == budget_group1['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == budget_group1['school_name'][2]) 
                                                  | (school_data_complete['school_name'] == budget_group1['school_name'][3])]
budget_group2_students = school_data_complete.loc[(school_data_complete['school_name'] == budget_group2['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == budget_group2['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == budget_group2['school_name'][2]) 
                                                  | (school_data_complete['school_name'] == budget_group2['school_name'][3])]
budget_group3_students = school_data_complete.loc[(school_data_complete['school_name'] == budget_group3['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == budget_group3['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == budget_group3['school_name'][2]) 
                                                  | (school_data_complete['school_name'] == budget_group3['school_name'][3])]
budget_group4_students = school_data_complete.loc[(school_data_complete['school_name'] == budget_group4['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == budget_group4['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == budget_group4['school_name'][2])]

avg_math_score_spend = [budget_group1_students['math_score'].mean(), 
                        budget_group2_students['math_score'].mean(), 
                        budget_group3_students['math_score'].mean(), 
                        budget_group4_students['math_score'].mean()]
avg_read_score_spend = [budget_group1_students['reading_score'].mean(), 
                        budget_group2_students['reading_score'].mean(), 
                        budget_group3_students['reading_score'].mean(), 
                        budget_group4_students['reading_score'].mean()]

pct_pass_read_spend = [budget_group1_students['read_PF'].value_counts()[1]/len(budget_group1_students)*100,
                       budget_group2_students['read_PF'].value_counts()[1]/len(budget_group2_students)*100,
                       budget_group3_students['read_PF'].value_counts()[1]/len(budget_group3_students)*100,
                       budget_group4_students['read_PF'].value_counts()[1]/len(budget_group4_students)*100]
pct_pass_math_spend = [budget_group1_students['math_PF'].value_counts()[1]/len(budget_group1_students)*100,
                       budget_group2_students['math_PF'].value_counts()[1]/len(budget_group2_students)*100,
                       budget_group3_students['math_PF'].value_counts()[1]/len(budget_group3_students)*100,
                       budget_group4_students['math_PF'].value_counts()[1]/len(budget_group4_students)*100]

bg1_student_both = budget_group1_students.loc[(budget_group1_students['math_PF'] == 'Pass') & (budget_group1_students['read_PF'] == 'Pass')]
bg2_student_both = budget_group2_students.loc[(budget_group2_students['math_PF'] == 'Pass') & (budget_group2_students['read_PF'] == 'Pass')]
bg3_student_both = budget_group3_students.loc[(budget_group3_students['math_PF'] == 'Pass') & (budget_group3_students['read_PF'] == 'Pass')]
bg4_student_both = budget_group4_students.loc[(budget_group4_students['math_PF'] == 'Pass') & (budget_group4_students['read_PF'] == 'Pass')]

pct_overall_spend = [len(bg1_student_both)/len(budget_group1_students)*100,
                    len(bg2_student_both)/len(budget_group2_students)*100,
                    len(bg3_student_both)/len(budget_group3_students)*100,
                    len(bg4_student_both)/len(budget_group4_students)*100]

# # Average math passing and reading passing percentages
# pct_overall_spend = [(pct_pass_read_spend[0] + pct_pass_math_spend[0])/2,
#                     (pct_pass_read_spend[1] + pct_pass_math_spend[1])/2,
#                     (pct_pass_read_spend[2] + pct_pass_math_spend[2])/2,
#                     (pct_pass_read_spend[3] + pct_pass_math_spend[3])/2]

scores_by_spending_df = pd.DataFrame({
                                        'Average Math Score': avg_math_score_spend,
                                        'Average Reading Score': avg_read_score_spend,
                                        '% Passing Math': pct_pass_math_spend,
                                        '% Passing Reading': pct_pass_read_spend,
                                        '% Overall Passing': pct_overall_spend
                                    }, index=spend_bin_labels)

scores_by_spending_df["% Passing Math"] = scores_by_spending_df["% Passing Math"].map("{:.2f}%".format)
scores_by_spending_df["% Passing Reading"] = scores_by_spending_df["% Passing Reading"].map("{:.2f}%".format)
scores_by_spending_df["% Overall Passing"] = scores_by_spending_df["% Overall Passing"].map("{:.2f}%".format)
scores_by_spending_df["Average Math Score"] = scores_by_spending_df["Average Math Score"].map("{:,.2f}".format)
scores_by_spending_df["Average Reading Score"] = scores_by_spending_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
<$584,83.36,83.96,93.70%,96.69%,90.64%
$585-629,79.98,82.31,79.11%,88.51%,70.94%
$630-644,77.82,81.30,70.62%,82.60%,58.84%
$645-675,77.05,81.01,66.23%,81.11%,53.53%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [26]:
size_bins = [0,999,1999,5000]
size_bin_labels = ['<1000', '1000-2000', '2000-5000']
school_data["size_groups"] = pd.cut(school_data["size"], size_bins, labels=size_bin_labels, include_lowest=True)

# create budget group dataframes
size_group1 = school_data.loc[school_data['size_groups'] == '<1000',:]
size_group2 = school_data.loc[school_data['size_groups'] == '1000-2000',:]
size_group3 = school_data.loc[school_data['size_groups'] == '2000-5000',:]
size_group1 = size_group1.reset_index(drop=True)
size_group2 = size_group2.reset_index(drop=True)
size_group3 = size_group3.reset_index(drop=True)

In [25]:
size_group1_students = school_data_complete.loc[(school_data_complete['school_name'] == size_group1['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == size_group1['school_name'][1])]
size_group2_students = school_data_complete.loc[(school_data_complete['school_name'] == size_group2['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == size_group2['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == size_group2['school_name'][2]) 
                                                  | (school_data_complete['school_name'] == size_group2['school_name'][3])
                                                  | (school_data_complete['school_name'] == size_group2['school_name'][4])]
size_group3_students = school_data_complete.loc[(school_data_complete['school_name'] == size_group3['school_name'][0]) 
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][1]) 
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][2]) 
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][3])
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][4]) 
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][5]) 
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][6])
                                                  | (school_data_complete['school_name'] == size_group3['school_name'][7])]

avg_math_score_size = [size_group1_students['math_score'].mean(), 
                        size_group2_students['math_score'].mean(), 
                        size_group3_students['math_score'].mean()]
avg_read_score_size = [size_group1_students['reading_score'].mean(), 
                        size_group2_students['reading_score'].mean(), 
                        size_group3_students['reading_score'].mean()]

pct_pass_read_size = [size_group1_students['read_PF'].value_counts()[1]/len(size_group1_students)*100,
                       size_group2_students['read_PF'].value_counts()[1]/len(size_group2_students)*100,
                       size_group3_students['read_PF'].value_counts()[1]/len(size_group3_students)*100]
pct_pass_math_size = [size_group1_students['math_PF'].value_counts()[1]/len(size_group1_students)*100,
                       size_group2_students['math_PF'].value_counts()[1]/len(size_group2_students)*100,
                       size_group3_students['math_PF'].value_counts()[1]/len(size_group3_students)*100]

sz1_student_both = size_group1_students.loc[(size_group1_students['math_PF'] == 'Pass') & (size_group1_students['read_PF'] == 'Pass')]
sz2_student_both = size_group2_students.loc[(size_group2_students['math_PF'] == 'Pass') & (size_group2_students['read_PF'] == 'Pass')]
sz3_student_both = size_group3_students.loc[(size_group3_students['math_PF'] == 'Pass') & (size_group3_students['read_PF'] == 'Pass')]

pct_overall_size = [len(sz1_student_both)/len(size_group1_students)*100,
                    len(sz2_student_both)/len(size_group2_students)*100,
                    len(sz3_student_both)/len(size_group3_students)*100]

# # Average math passing and reading passing percentages
# pct_overall_size = [(pct_pass_read_size[0] + pct_pass_math_size[0])/2,
#                     (pct_pass_read_size[1] + pct_pass_math_size[1])/2,
#                     (pct_pass_read_size[2] + pct_pass_math_size[2])/2]

scores_by_size_df = pd.DataFrame({
                                        'Average Math Score': avg_math_score_size,
                                        'Average Reading Score': avg_read_score_size,
                                        '% Passing Math': pct_pass_math_size,
                                        '% Passing Reading': pct_pass_read_size,
                                        '% Overall Passing': pct_overall_size
                                    }, index=size_bin_labels)

scores_by_size_df["% Passing Math"] = scores_by_size_df["% Passing Math"].map("{:.2f}%".format)
scores_by_size_df["% Passing Reading"] = scores_by_size_df["% Passing Reading"].map("{:.2f}%".format)
scores_by_size_df["% Overall Passing"] = scores_by_size_df["% Overall Passing"].map("{:.2f}%".format)
scores_by_size_df["Average Math Score"] = scores_by_size_df["Average Math Score"].map("{:,.2f}".format)
scores_by_size_df["Average Reading Score"] = scores_by_size_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
<1000,83.83,83.97,93.95%,96.04%,90.14%
1000-2000,83.37,83.87,93.62%,96.77%,90.62%
2000-5000,77.48,81.20,68.65%,82.13%,56.57%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [28]:
school_data_complete['type'].value_counts()

District    26976
Charter     12194
Name: type, dtype: int64

In [32]:
type_group2_students = school_data_complete.loc[(school_data_complete['type'] == 'District')]
type_group1_students = school_data_complete.loc[(school_data_complete['type'] == 'Charter')]

avg_math_score_type = [type_group1_students['math_score'].mean(), 
                        type_group2_students['math_score'].mean()]
avg_read_score_type = [type_group1_students['reading_score'].mean(), 
                        type_group2_students['reading_score'].mean()]

pct_pass_read_type = [type_group1_students['read_PF'].value_counts()[1]/len(type_group1_students)*100,
                       type_group2_students['read_PF'].value_counts()[1]/len(type_group2_students)*100]
pct_pass_math_type = [type_group1_students['math_PF'].value_counts()[1]/len(type_group1_students)*100,
                       type_group2_students['math_PF'].value_counts()[1]/len(type_group2_students)*100]

ty1_student_both = type_group1_students.loc[(type_group1_students['math_PF'] == 'Pass') & (type_group1_students['read_PF'] == 'Pass')]
ty2_student_both = type_group2_students.loc[(type_group2_students['math_PF'] == 'Pass') & (type_group2_students['read_PF'] == 'Pass')]

pct_overall_type = [len(ty1_student_both)/len(type_group1_students)*100,
                    len(ty2_student_both)/len(type_group2_students)*100]

# # Average math passing and reading passing percentages
# pct_overall_type = [(pct_pass_read_type[0] + pct_pass_math_type[0])/2,
#                     (pct_pass_read_type[1] + pct_pass_math_type[1])/2]

scores_by_type_df = pd.DataFrame({
                                        'Average Math Score': avg_math_score_type,
                                        'Average Reading Score': avg_read_score_type,
                                        '% Passing Math': pct_pass_math_type,
                                        '% Passing Reading': pct_pass_read_type,
                                        '% Overall Passing': pct_overall_type
                                    }, index=['Charter', 'District'])

scores_by_type_df["% Passing Math"] = scores_by_type_df["% Passing Math"].map("{:.2f}%".format)
scores_by_type_df["% Passing Reading"] = scores_by_type_df["% Passing Reading"].map("{:.2f}%".format)
scores_by_type_df["% Overall Passing"] = scores_by_type_df["% Overall Passing"].map("{:.2f}%".format)
scores_by_type_df["Average Math Score"] = scores_by_type_df["Average Math Score"].map("{:,.2f}".format)
scores_by_type_df["Average Reading Score"] = scores_by_type_df["Average Reading Score"].map("{:,.2f}".format)
scores_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Charter,83.41,83.90,93.70%,96.65%,90.56%
District,76.99,80.96,66.52%,80.91%,53.70%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
